# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
print(sum)

8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [7]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

## Question 1: What is the sum of the outputs of the generator for limit = 5?

In [8]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
print(sum)

8.382332347441762


## Question 2: What is the 13th number yielded by the generator?

In [16]:
limit = 13
generator = square_root_generator(limit)
gen = list(generator)
print(gen[-1])

3.605551275463989


## Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [17]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# load generator people_1() to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(), table_name="question_3", write_disposition="append")
print(info)

# load generator people_2() to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(), table_name="question_3", write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708185798.979599 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708185799.8777313 is LOADED and contains no failed jobs


In [22]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print("Loaded tables: ")
display(conn.sql("show tables"))

print("\n Question 3 table below:")
total_ages = conn.sql("SELECT sum(age) FROM generators.question_3")
display(total_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question_3          │
└─────────────────────┘


 Question 3 table below:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

## Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [29]:
merge_pipeline = dlt.pipeline(destination="duckdb", dataset_name="generators")
info = merge_pipeline.run(
    people_1(),
    table_name="question_4",
    write_disposition="merge",
    primary_key="id",
)
print(info)
info = merge_pipeline.run(
    people_2(),
    table_name="question_4",
    write_disposition="merge",
    primary_key="id",
)
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708186512.0167751 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708186512.768593 is LOADED and contains no failed jobs


In [30]:
conn = duckdb.connect(f"{merge_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{merge_pipeline.dataset_name}'")
print("Loaded tables: ")
display(conn.sql("show tables"))

total_ages = conn.sql("SELECT * FROM generators.question_4")
display(total_ages)

print("\n Question 4 table below:")
total_ages = conn.sql("SELECT sum(age) FROM generators.question_4")
display(total_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ question_3          │
│ question_4          │
└─────────────────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708186512.0167751 │ 8lQbME5iEVF6gg │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708186512.0167751 │ 6PI3U9AOfvqphw │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1708186512.768593  │ rkg/r4kxfxInVA │ Job_8      │
│     4 │ Person_4 │    34 │ City_B  │ 1708186512.768593  │ n4YH+bHA6yfCYw │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1708186512.768593  │ 0FEXUqdWQLRuZw │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1708186512.768593  │ 2OMmWblOoqp7mQ │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1708186512.768593  │ 6zxl0FtdiMI8DQ │ Job_3      │
│     6 │ 


 Question 4 table below:


┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX